In [ ]:
import pandas as pd
df = pd.read_csv("df_neurips_limitation_and_OR_with_cited_data.csv")

In [ ]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
!pip3 -q install langchain
# For the chat-style client
from langchain.chat_models import ChatOpenAI

# If you also need the OpenAI LLM wrapper
from langchain.llms import OpenAI
!pip install --upgrade langchain
import time
import os
import logging
import sys
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
import io
import sys
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from langchain.schema import Document as LCDocument
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
# New imports for FAISS and BM25 from LangChain
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever
from langchain.schema import Document as LCDocument
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.document_loaders import CSVLoader
from langchain.text_splitter    import CharacterTextSplitter
from langchain.embeddings       import HuggingFaceEmbeddings
from langchain.vectorstores     import FAISS
from langchain.retrievers       import BM25Retriever
from sklearn.metrics.pairwise  import cosine_similarity
import numpy as np
# from llama_index import Document as LIDoc, VectorStoreIndex
from llama_index.core.schema import Document as LIDoc

from langchain.embeddings import HuggingFaceEmbeddings
# from llama_index.node_parser import SentenceSplitter
from llama_index.embeddings.langchain import LangchainEmbedding
# from langchain.embeddings import HuggingFaceEmbeddings
import io
import sys

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores    import FAISS
from langchain.retrievers      import BM25Retriever, EnsembleRetriever
from langchain.schema import SystemMessage, HumanMessage

# 2) Import the predictor and context from their true locations
from llama_index.core.service_context import ServiceContext
from llama_index.core.service_context_elements.llm_predictor import LLMPredictor


In [ ]:
import tiktoken

# 1) choose your model’s max tokens
MAX_CONTEXT_TOKENS = 127_000
MODEL_NAME         = "gpt-4o-mini"

# 2) helper to count tokens of any string
def count_tokens(text: str, model: str = MODEL_NAME) -> int:
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

# 3) helper to truncate a list of passages so that
#    len(query) + sum(len(passages)) <= MAX_CONTEXT_TOKENS
def truncate_for_context(
    query: str,
    passages: list[str],
    max_tokens: int = MAX_CONTEXT_TOKENS,
    model: str = MODEL_NAME,
) -> list[str]:
    enc        = tiktoken.encoding_for_model(model)
    q_tokens   = enc.encode(query)
    remaining  = max_tokens - len(q_tokens)
    kept, used = [], 0

    for p in passages:
        p_tokens = enc.encode(p)
        if used + len(p_tokens) > remaining:
            # only take as many tokens as fit
            allowed = remaining - used
            if allowed > 0:
                kept.append(enc.decode(p_tokens[:allowed]))
            break
        kept.append(p)
        used += len(p_tokens)

    return kept

In [ ]:
# single embeddings object
# hf_emb = HuggingFaceEmbeddings("sentence-transformers/all-MiniLM-L6-v2")

def make_retriever_for_docs(docs, k=3):
    # FAISS
    faiss_store = FAISS.from_documents(docs, hf_emb)
    faiss_r     = faiss_store.as_retriever(search_kwargs={"k": k})

    # BM25
    bm25_r      = BM25Retriever.from_documents(docs)
    bm25_r.k    = k

    # ensemble (50/50 weight)
    return EnsembleRetriever(
        retrievers=[faiss_r, bm25_r],
        weights=[0.5, 0.5]
    )


generated_limitations = []
from langchain.chat_models import ChatOpenAI
from langchain.chains      import RetrievalQA

# your GPT-4o-mini chat model
chat_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def generate_limitations(question, retriever):
    qa = RetrievalQA.from_chain_type(
        llm=chat_llm,
        chain_type="stuff",        # simplest “stuff the docs into the prompt”
        retriever=retriever,
        return_source_documents=False
    )
    return qa.run(question).strip()



/tmp/ipykernel_652548/2005548234.py:25: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
# define the (label, column_name) pairs in order
fields = [
    ("Abstract",                "df_Abstract"),
    ("Introduction",            "df_Introduction"),
    ("Related Work",            "df_Related_Work"),
    ("Methodology",             "df_Methodology"),
    ("Dataset",                 "df_Dataset"),
    ("Conclusion",              "df_Conclusion"),
    ("Experiment and Results",  "df_Experiment_and_Results"),
]

def make_response_string(row):
    parts = []
    for label, col in fields:
        text = row.get(col, "")
        if isinstance(text, str) and text.strip():
            parts.append(f"{label}: {text.strip()}")
    # join with a separator of your choice
    return " | ".join(parts)

# create the new column
df["response_string_all"] = df.apply(make_response_string, axis=1)


In [ ]:
from langchain.schema import SystemMessage, HumanMessage


system_prompt = '''You are a helpful, respectful, and honest assistant for generating limitations or shortcomings of a research paper.
 Generate limitations or shortcomings for the following passages from the scientific paper.'''

# ─── your existing setup ─────────────────────────────────────────────────────
chat_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ─── new unified helper ───────────────────────────────────────────────────────
def run_critic(prompt: str,*,system_prompt: str | None = None) -> str:
    """
    Wraps ChatOpenAI to (optionally) send a system prompt + your user prompt,
    and returns the assistant's reply as a stripped string.
    """
    messages: list[SystemMessage | HumanMessage] = []

    if system_prompt:
        messages.append(SystemMessage(content=system_prompt))

    messages.append(HumanMessage(content=prompt))

    # this will wait for the full response (no streaming)
    response = chat_llm.invoke(messages)
    return response.content.strip()

In [ ]:

def count_tokens(text: str, model: str = MODEL_NAME) -> int:
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text, disallowed_special=()))

def truncate_for_context(query: str,passages: list[str],
    max_tokens: int = MAX_CONTEXT_TOKENS,
    model: str = MODEL_NAME,
) -> list[str]:
    enc       = tiktoken.encoding_for_model(model)
    # allow all specials
    q_tokens  = enc.encode(query, disallowed_special=())
    budget    = max_tokens - len(q_tokens)
    kept, used = [], 0
    for p in passages:
        p_toks = enc.encode(p, disallowed_special=())
        if used + len(p_toks) > budget:
            if budget - used > 0:
                kept.append(enc.decode(p_toks[:(budget - used)]))
            break
        kept.append(p)
        used += len(p_toks)
    return kept


def ensure_passages_within_budget(
    query: str,
    passages: list[str],
    max_tokens: int = MAX_CONTEXT_TOKENS,
    model: str = MODEL_NAME,
) -> list[str]:
    """
    Returns `passages` truncated so that
    count_tokens(query + concat(passages)) ≤ max_tokens.
    """
    # count full size
    total = count_tokens(query + "\n\n".join(passages), model=model)
    if total <= max_tokens:
        return passages

    print(f"Truncating context ({total} tokens)…")
    # truncate_for_context only returns the shorter passages list
    return truncate_for_context(query, passages, max_tokens=max_tokens, model=model)


In [ ]:
import os
from openai import OpenAI
import time

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Define which sections to pull in for every row
SECTIONS = ["df_Abstract", "df_Introduction", "df_Related_Work",
            "df_Methodology", "df_Dataset", "df_Experiment_and_Results", "df_Conclusion"]

# Define a helper that streams one prompt and returns the full text
def run_critic(prompt: str) -> str:
    summary = ""
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
        temperature=0
    )
    for chunk in stream:
        summary += chunk.choices[0].delta.content or ""
    return summary.strip()

generated_limitations = []

for idx in range(10): # len(df_lim)
    # build a list of “Section:\ncontent” strings
    pieces = []
    for sec in SECTIONS:
        col = sec.replace(" ", "_")    # e.g. "Related Work" → "Related_Work"
        if pd.notna(df.loc[idx, col]):
            pieces.append(f"{sec}:\n{df.loc[idx, col]}")

    text_blob = "\n\n".join(pieces)

    extractor_agent = run_critic(
        '''You are an expert in scientific literature analysis. Your task is to carefully read the provided scientific article and
        extract all explicitly stated limitations as mentioned by the authors. Focus on sections such as the Discussion, Conclusion.
        List each limitation verbatim, including direct quotes where possible, and provide
        a brief context (e.g., what aspect of the study the limitation pertains to). Ensure accuracy and avoid inferring or adding
        limitations not explicitly stated. If no limitations are mentioned, state this clearly. Output your findings in a structured
        format with bullet points.\n\n'''
        + text_blob
    )
    analyzer_agent = run_critic(
        '''You are a critical scientific reviewer with expertise in research methodology and analysis. Your task is to analyze the
        provided scientific article and identify potential limitations that are not explicitly stated by the authors. Focus on aspects
        such as study design, sample size, data collection methods, statistical analysis, scope of findings, and underlying assumptions.
        For each inferred limitation, provide a clear explanation of why it is a limitation and how it impacts the study’s validity,
        reliability, or generalizability. Ensure your inferences are grounded in the article’s content and avoid speculative assumptions.
        Output your findings in a structured format with bullet points, including a brief justification for each limitation.\n\n'''
        + text_blob
    )
    reviewer_agent = run_critic(
        '''You are an expert in open peer review with a focus on transparent and critical evaluation of scientific research. Your task
        is to review the provided scientific article from the perspective of an external peer reviewer. Identify potential limitations
        that might be raised in an open review process, considering common critiques such as reproducibility, transparency,
        generalizability, or ethical considerations. If possible, leverage insights from similar studies or common methodological
        issues in the field (search the web or X posts if needed for context). For each limitation, explain why it would be a
        concern in an open review and how it aligns with peer review standards. Output your findings in a structured format with
        bullet points, ensuring each limitation is relevant to the article’s content.:\n\n'''
        + text_blob
    )

    # 3) master coordinator to fuse them
    coord_prompt = (
    '''You are a **Master Coordinator**. You are an expert in scientific communication and synthesis.
    Your task is to integrate the limitations provided by three other agents:
    1. The **Extractor** (explicit limitations from the article),
    2. The **Analyzer** (inferred limitations from critical analysis),
    3. The **Reviewer** (limitations from an open review perspective).

    Your goals are to:
    1. Combine all limitations into a cohesive list, removing redundancies.
    2. Ensure each limitation is clearly stated, scientifically valid, and aligned with the article’s content.
    3. Prioritize limitations explicitly mentioned by the authors, supplementing them with inferred or peer-review-based limitations only if they add value.
    4. Highlight discrepancies between the agents’ outputs and resolve them by cross-referencing the article.
    5. Format the final list in a clear, concise, and professional manner, suitable for inclusion in a scientific review or report.

    Provide a brief justification for each limitation, noting its source:
    - "Author-stated" (from Extractor),
    - "Inferred" (from Analyzer),
    - "Peer-review-derived" (from Reviewer).

    You will be given three lists of sub-limitations:\n\n'''
    f"**Extractor Agent**:\n{extractor_agent}\n\n"
    f"**Analyzer Agent**:\n{analyzer_agent}\n\n"
    f"**Reviewer Agent**:\n{reviewer_agent}\n\n"
    )
    final_lim = run_critic(coord_prompt)

    generated_limitations.append({
        "Extractor": extractor_agent,
        "Analzyer": analyzer_agent,
        "Reviewer": reviewer_agent,
        "final": final_lim
    })


In [ ]:
# remove future work
import re

def remove_future_entries(entries):
    """
    Given a list of lists (where each sub-list contains strings),
    return a new list omitting sublists where the first item contains
    'future' inside **double asterisks** (case-insensitive).
    """
    filtered = []
    for sublist in entries:
        if isinstance(sublist, list) and len(sublist) > 0:
            first_item = sublist[0]
            # Check if 'future' appears inside **...** (case-insensitive)
            if not re.search(r'\*\*.*future.*\*\*', first_item, re.IGNORECASE):
                filtered.append(sublist)
        else:
            filtered.append(sublist)  # Keep non-list entries as-is
    return filtered

# Apply to the DataFrame column
df_generated_limitations_2["generated_limitations_1"] = df_generated_limitations_2["generated_limitations_1"].apply(
    lambda lst: remove_future_entries(lst) if isinstance(lst, list) else lst
)


# remove future work
import re

def remove_future_entries(entries):
    """
    Given a list of lists (where each sub-list contains strings),
    return a new list omitting sublists where the first item contains
    'future' inside **double asterisks** (case-insensitive).
    """
    filtered = []
    for sublist in entries:
        if isinstance(sublist, list) and len(sublist) > 0:
            first_item = sublist[0]
            # Check if 'future' appears inside **...** (case-insensitive)
            if not re.search(r'\*\*.*future.*\*\*', first_item, re.IGNORECASE):
                filtered.append(sublist)
        else:
            filtered.append(sublist)  # Keep non-list entries as-is
    return filtered

# Apply to the DataFrame column
df["Lim_and_OR_ground_truth_final"] = df["Lim_and_OR_ground_truth_final"].apply(
    lambda lst: remove_future_entries(lst) if isinstance(lst, list) else lst
)

def remove_here_are_the(entries):
    """
    Given a list of lists (where each sub-list contains strings),
    return a new list omitting any sub-list that starts with "1. Here are the"
    (case-insensitive and whitespace-tolerant).
    """
    filtered = []
    for sublist in entries:
        if isinstance(sublist, list) and len(sublist) > 0:
            first_item = sublist[0].strip().lower()  # Clean whitespace and lowercase
            if not first_item.startswith("1. here are the"):
                filtered.append(sublist)
        else:
            filtered.append(sublist)  # Keep non-list entries as-is
    return filtered

# Apply to every row in the DataFrame
df["Lim_and_OR_ground_truth_final"] = df["Lim_and_OR_ground_truth_final"].apply(
    lambda lst: remove_here_are_the(lst) if isinstance(lst, list) else lst
)